In [1]:
import os
import re
import json
import sys
from glob import glob
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import rcParams
from matplotlib import pyplot as plt

sys.path.append('..', '..')

from src.melody import Melody
from src.utils import notes_to_midi, notes_and_chord_to_midi, get_chord_progressions

rcParams['figure.figsize'] = 20,10

In [2]:
import importlib
import objective_metrics


importlib.reload(objective_metrics)

<module 'objective_metrics' from '/media/manu/Data/PycharmProjects/thesis/src/objective_metrics.py'>

In [3]:
version = '1.2'
folder = f'../data/generated/22_05_22_00_52_58_transpose_all_chord_extended_7_batchsize_128_seed_1234567890/seen/'
filepaths = [y for x in os.walk(folder) for y in glob(os.path.join(x[0], '*.csv'))]

chord_progressions = get_chord_progressions('..')

In [4]:
filepaths[0]

'../data/generated/22_05_22_00_52_58_transpose_all_chord_extended_7_batchsize_128_seed_1234567890/seen/22_05_26_13_56_57 Softly As In A Morning Sunrise.csv'

In [5]:
metrics = {}

# filepath = '../data/split_melody/Real Book/All Of Me_original.csv'

all_dfs = []

for filepath in filepaths:
    df = pd.read_csv(filepath, index_col=0)
    
    df['measure'] = (df['ticks'] // 48).astype('int')
    df = df.rename({
        'improvised_pitch': 'pitch', 
        'improvised_duration': 'duration'
    }, axis=1)
    df['offset'] = df['offset'].astype('int') 
    df['ticks'] = df['ticks'].astype('int') 
    df['duration'] = df['duration'].astype('int') 
    df['pitch'] = df['pitch'].replace(np.nan, -1).astype('int') 
    
    all_dfs.append(df)

    # PCHE
    pe1 = objective_metrics.compute_piece_pitch_entropy(df, 1)
    pe4 = objective_metrics.compute_piece_pitch_entropy(df, 4)
    
    # PV
    pv4 = objective_metrics.calculate_PV(df[df['pitch'] >= 0], 4)
    
    # TS
    ts8 = objective_metrics.calculate_TS(df, 8)
    
    # CPR
    cpr2 = objective_metrics.calculate_CPR(df, 2)
    
    # ----------
    
    # DPR
    dpr12 = objective_metrics.calculate_DPR(df, 12)
    
    # GPS
    gs = objective_metrics.compute_piece_groove_similarity(df, max_pairs=np.inf)
    
    # RV
    rv4 = objective_metrics.calculate_RV(df, 4)
    
    # QR
    qr = objective_metrics.calculate_QD(df)
    
    # ----------
    
    # HC
    hc = objective_metrics.calculate_HC(df[df['pitch'] >= 0])
    
    # ----------
    
    # RN 
#     rm = objective_metrics.calculate_RM(df)

    metrics[filepath] = {
        'PCHE1': pe1,
        'PCHE4': pe4,
        'PV4': pv4,
        'TS8': ts8,
        'CPR2': cpr2,
        'DPR12': dpr12,       
        'GPS': gs,
        'RV4': rv4,
        'QR': qr,
        'HC': hc,
#         'RM': rm
    }

total_df = pd.concat(all_dfs)  

10.620975048436001 0.36624051891158627


TypeError: object of type 'float' has no len()

In [ ]:
gen = pd.DataFrame().from_dict(metrics).T
gen

In [ ]:
gen['HC-m'] = gen['HC'].apply(np.mean)

for metric in ['PCHE1', 'PCHE4', 'PV4', 'TS8', 'CPR2', 'DPR12', 'GPS', 'RV4', 'QR', 'HC-m']:
    print(f'{metric} - {gen[metric].mean():5.2f} - {gen[metric].std():5.2f}')

In [ ]:
gen['PV4']